In [32]:
from keras.models import Model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda, Concatenate, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from utils import decode_netout, compute_overlap, compute_ap
from keras.applications.mobilenet import MobileNet
from keras.layers.merge import concatenate
from keras.optimizers import SGD, Adam, RMSprop
from preprocessing import BatchGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from backend import TinyYoloFeature, MobileNetFeature, SqueezeNetFeature

import tensorflow as tf
import numpy as np
import os
import cv2

In [45]:
class Vehicle(object):
    def __init__(self, backend,
                       input_size, 
                       labels,
                       max_box_per_image=50,
                       anchors=[0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]):

        self.input_size = input_size        
        self.labels = list(labels)
        self.nb_class = len(self.labels)
        self.nb_box = len(anchors)//2
        self.class_wt = np.ones(self.nb_class, dtype='float32')
        self.anchors = anchors

        self.max_box_per_image = max_box_per_image

        ##########################
        # Make the model
        ##########################

        # make the feature extractor layers
        input_image     = Input(shape=(self.input_size, self.input_size, 3))
        self.true_boxes = Input(shape=(1, 1, 1, max_box_per_image , 4))  

        if backend == 'SqueezeNet':
            self.feature_extractor = SqueezeNetFeature(self.input_size)        
        elif backend == 'MobileNet':
            self.feature_extractor = MobileNetFeature(self.input_size)
        elif backend == 'Tiny Yolo':
            self.feature_extractor = TinyYoloFeature(self.input_size)
        else:
            raise Exception('Architecture not supported! Only support Tiny Yolo, MobileNet, SqueezeNet at the moment!')

        print(self.feature_extractor.get_output_shape())    
        self.grid_h, self.grid_w = self.feature_extractor.get_output_shape()        
        features = self.feature_extractor.extract(input_image)            

        # make the object detection layer
        output = Conv2D(self.nb_box * (4 + 1 + self.nb_class), 
                        (1,1), strides=(1,1), 
                        padding='same', 
                        name='DetectionLayer', 
                        kernel_initializer='lecun_normal')(features)
        output = Reshape((self.grid_h, self.grid_w, self.nb_box, 4 + 1 + self.nb_class))(output)
        output = Lambda(lambda args: args[0])([output, self.true_boxes])

        
        
        convdir  = Conv2D(2, (3, 3), activation='relu', padding='same', use_bias=False, name='dir1')(input_image)
        convdir2 = Conv2D(2, (3, 3), activation='relu', padding='same', use_bias=False, name='dir2')(convdir)
        pooldir = MaxPooling2D(pool_size=2, name='dir3')(convdir2)
        
        convdir1  = Conv2D(4, (3, 3), activation='relu', padding='same', use_bias=False, name='dir4')(pooldir)
        convdir12 = Conv2D(4, (3, 3), activation='relu', padding='same', use_bias=False, name='dir5')(convdir1)
        pooldir1 = AveragePooling2D(pool_size=2, name='dir6')(convdir12)
        
        flat1 = Flatten()(pooldir1)
        flat2 = Flatten()(output)
        
        added = Concatenate()([flat1, flat2])
        
        #fc1 = Dense(32, activation='relu', name='fchingona')(added)
        fc2 = Dense(6, activation='relu', name='stemen')(added)
        
        self.model = Model([input_image, self.true_boxes], [output, fc2])

        # print a summary of the whole model
        self.model.summary()

In [46]:
v = Vehicle('SqueezeNet',
     256,
     ["left_arrow", "person", "right_arrow", "sem", "stop"],
     anchors=[0.26,0.80, 0.29,0.40, 0.37,0.52, 0.46,0.63, 0.55,0.79, 0.57,1.87, 0.64,1.06, 0.99,1.52, 1.03,2.73, 1.52,4.59])

(15, 15)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_58 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
dir1 (Conv2D)                   (None, 256, 256, 2)  54          input_58[0][0]                   
__________________________________________________________________________________________________
dir2 (Conv2D)                   (None, 256, 256, 2)  36          dir1[0][0]                       
__________________________________________________________________________________________________
model_41 (Model)                (None, 15, 15, 512)  722496      input_58[0][0]                   
__________________________________________________________________________________________________
d

In [29]:
v.model